# KD_TREE Based

In [92]:
import numpy as np
import time
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist

def calculate_I1_i(array,h):
    s = time.time()
    distances = cdist(array,array, metric='euclidean')**2
    sum_i =  np.sum(np.exp(-distances/(4*h**2)))
    
    return sum_i,time.time()-s#对于一条数据，计算距离矩阵的时间

def gaussian_kernel(array,h,d):
    d1 = (np.sum(array**2,axis=1)/(2*h**2))

    kernel_values = (1 / (np.sqrt(2 * np.pi) * h))**d * np.exp(-d1)
    return np.sum(kernel_values)  
    
def calculate_I2_i(array,zi,h,d):

    inside = (zi-array).reshape(-1,d)

    return gaussian_kernel(inside,h,d)
    


def bootstrap_knn(data, row_index, s_ratio, k, B, missing_mask, complete_mask):

    n, _ = data.shape
    s = int(round((n-1)*s_ratio))
    if s<k:
        k = s
        warnings.warn("s_complete < n_neighbors, replacing n_neighbors with s_complete", UserWarning)
    neighbors_data = np.zeros((B * k, len(missing_mask))) 
    query_point = data[row_index,complete_mask].reshape(1,-1)
    start_time = time.time()
    for i in range(B):
        indices_1 = np.random.choice(np.delete(np.arange(n), row_index), size=s, replace=False)

        subsample = data[indices_1][:,complete_mask].copy()
        
        nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(subsample)
        _, indices = nbrs.kneighbors(query_point)
        
        neighbor_indices = indices[0]

        neighbors_data[i * k: (i + 1) * k] = data[indices_1][neighbor_indices][:,missing_mask]
    x_time = time.time()-start_time#对于一条数据来说，找到B*k个neighbours花费的时间
    return neighbors_data,x_time

                
def calculate_score_restricted_1(data, s_ratio, k, B,h, missing_mask, complete_mask):
    n,d = data.shape
    I1 = []
    I2 = []
    B_time = []
    D_time=[]
    s_time = []
    if n >= 3000:
        for i in np.random.choice(range(n),3000,replace  =False):
            start_time = time.time()
            X,b_time = bootstrap_knn(data,i, s_ratio, k, B, missing_mask, complete_mask)
            B_time.append(b_time)
            I1_1,d_time = calculate_I1_i(X,h)
            D_time.append(d_time)
            I1_i = (1/(2*h*np.pi**0.5))**len(missing_mask)*(1/(3000*B**2*k**2))*I1_1
            I2_i = (1/(B*k*3000))*calculate_I2_i(X,data[i,missing_mask],h,len(missing_mask))
            s_time.append(time.time()-start_time)
            I1.append(I1_i)
            I2.append(I2_i)
    else:
        for i in range(n):
            start_time = time.time()
            X,b_time = bootstrap_knn(data,i, s_ratio, k, B, missing_mask, complete_mask)
            B_time.append(b_time)
            I1_1,d_time = calculate_I1_i(X,h)
            D_time.append(d_time)
            I1_i = (1/(2*h*np.pi**0.5))**len(missing_mask)*(1/(3000*B**2*k**2))*I1_1
            I2_i = (1/(B*k*3000))*calculate_I2_i(X,data[i,missing_mask],h,len(missing_mask))
            s_time.append(time.time()-start_time)
            I1.append(I1_i)
            I2.append(I2_i)
    print('I1',sum(I1),'I2',sum(I2))
    return sum(I1)-2*sum(I2),B_time,D_time,s_time


In [68]:
#generate data
def normalization(data):
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    ranges = max_vals - min_vals
    normalized_data = (data - min_vals) / ranges
    return normalized_data, min_vals, ranges
def generate_normalized_data(num_samples,vector):
    u = np.random.uniform(0,1,(num_samples,vector))+np.random.normal(0, 0.05,(num_samples,vector))
    x = 4*np.pi*u
    y = np.sum(np.sin(x),axis=1).reshape(-1,1)+np.random.normal(0,0.5,(num_samples,1))
    data = np.concatenate((x,y),axis=1)
    normalized_data,_,_ = normalization(data) 
    return normalized_data

In [93]:
data = generate_normalized_data(1000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 15,50,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 0.4042814086380018 I2 0.45428526733705427
the time for finding B*K the neighbours 66.59325790405273
the time for calculating the distance 25.441046237945557
the time for computing score 92.86803603172302
finding neighbours/all 0.7170740412912897
computing distance/all 0.273948360760586


In [69]:
#sample size (3000,3),B=5,s=0.3,k=10
data = generate_normalized_data(3000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.3, 5, 10,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.261366313071258 I2 1.366469912896041
the time for finding B*K the neighbours 46.23117017745972
the time for calculating the distance 0.39989447593688965
the time for computing score 46.98595571517944
finding neighbours/all 0.9839359330627411
computing distance/all 0.008510936296815569


In [70]:
#sample size (3000,3),B=15,s=0.7,k=60
data = generate_normalized_data(3000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 15, 60,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.2060856799076407 I2 1.3474131471707522
the time for finding B*K the neighbours 469.65077352523804
the time for calculating the distance 111.79019212722778
the time for computing score 584.7179358005524
finding neighbours/all 0.8032091112139857
computing distance/all 0.19118652820897816


In [71]:
#sample size (3000,7),B=15,s=0.7,k=60
data = generate_normalized_data(3000,6)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 15, 60,0.1, np.array([0]),np.array([1,2,3,4,5,6]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.1077052537118843 I2 1.1067693922662944
the time for finding B*K the neighbours 872.8903737068176
the time for calculating the distance 111.93913698196411
the time for computing score 988.1524515151978
finding neighbours/all 0.8833559764673544
computing distance/all 0.11328124198884558


In [72]:
#sample size (3000,3),B=15,s=0.7,k=10
data = generate_normalized_data(3000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 15,10,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.2432536035967532 I2 1.3718912845906912
the time for finding B*K the neighbours 74.8816819190979
the time for calculating the distance 1.9583923816680908
the time for computing score 77.19596695899963
finding neighbours/all 0.9700206483438325
computing distance/all 0.025369102283649525


In [73]:
#sample size (3000,3), B=15,s=0.1,k=60
data = generate_normalized_data(3000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.1, 15,60,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.1470215340342613 I2 1.3311181934268788
the time for finding B*K the neighbours 206.8263885974884
the time for calculating the distance 111.5419442653656
the time for computing score 321.6292414665222
finding neighbours/all 0.6430584099083434
computing distance/all 0.3468028707737253


In [74]:
#sample size (3000,3), B=5,s=0.7,k=60
data = generate_normalized_data(3000,2)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 5,60,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 1.3686607598897542 I2 1.383036812355105
the time for finding B*K the neighbours 466.65701961517334
the time for calculating the distance 9.311933279037476
the time for computing score 477.1795103549957
finding neighbours/all 0.9779485696441698
computing distance/all 0.01951452876111529


In [75]:
#sample size (3000,10),B=15,s=0.7,k=60，缺失3维
data = generate_normalized_data(3000,9)
score,B_time,D_time,s_time = calculate_score_restricted_1(data,0.7, 15, 60,0.1, np.array([0,1,2]),np.array([3,4,5,6,7,8,9]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',sum(D_time))
print('the time for computing score',sum(s_time))
print('finding neighbours/all',sum(B_time)/sum(s_time))
print('computing distance/all',sum(D_time)/sum(s_time))

I1 2.0809129596461258 I2 1.3632182414887937
the time for finding B*K the neighbours 972.2258670330048
the time for calculating the distance 115.9160566329956
the time for computing score 1091.4878783226013
finding neighbours/all 0.8907344610433252
computing distance/all 0.1062000402708415


# Based on Distance

In [88]:
import numpy as np
import time
from scipy.spatial.distance import cdist
def calculate_distances(data, complete_mask):
    """Calculate pairwise distances for the given mask"""
    # Select the dimensions according to the mask
    selected_data = data[:, complete_mask]
    # Calculate pairwise distances using Euclidean distance
    distance_matrix = cdist(selected_data, selected_data, metric='euclidean')
    return distance_matrix

def k_nearest_neighbors(row_index, B, distance_matrix, s_ratio, k,data,miss_mask):
    """Find k nearest neighbors from s randomly selected samples"""
    num_samples = distance_matrix.shape[0]
    s = int(s_ratio*num_samples)
    neighbors_data = np.zeros((B*k,len(miss_mask)))
    
    miss_mask_data = data[:,miss_mask]
    # Exclude the current sample from the rest
    rest_indices = list(range(num_samples))
    rest_indices.remove(row_index)
    st = time.time()
    for b in range(B):

        # Randomly select s samples from the rest
        selected_indices = np.random.choice(rest_indices, s, replace=False)
        
        # Calculate the distances from the current sample to the selected samples
        distances = distance_matrix[row_index, selected_indices]
        
        # Get the indices of the k nearest neighbors
        k_nearest_indices = selected_indices[np.argsort(distances)[:k]]
        neighbors_data[b * k: (b + 1) * k] =  miss_mask_data[k_nearest_indices]
        
    return neighbors_data,time.time()-st





def calculate_I1_i(array,h):
    distances = cdist(array,array, metric='euclidean')**2
    sum_i =  np.sum(np.exp(-distances/(4*h**2)))
    
    return sum_i

def gaussian_kernel(array,h,d):
    d1 = (np.sum(array**2,axis=1)/(2*h**2))

    kernel_values = (1 / (np.sqrt(2 * np.pi) * h))**d * np.exp(-d1)
    return np.sum(kernel_values)  
    
def calculate_I2_i(array,zi,h,d):

    inside = (zi-array).reshape(-1,d)

    return gaussian_kernel(inside,h,d)


def calculate_score_restricted(data, s_ratio, k, B,h, missing_mask, complete_mask):
    n,d = data.shape
    I1 = []
    I2 = []
    B_time = []
    D_time=[]
    st_1= time.time()
    if n >= 3000:
        st = time.time()
        selected_indices = np.random.choice(range(n),3000,replace  =False)
        data_selected = data[selected_indices]
        distance_matrix = calculate_distances(data_selected, complete_mask)
        d_time = time.time()-st
        for i in range(3000):
            X,b_time = k_nearest_neighbors(i, B, distance_matrix, s_ratio, k ,data_selected ,missing_mask)

            B_time.append(b_time)
            
            I1_1 = calculate_I1_i(X,h)
            I1_i = (1/(2*h*np.pi**0.5))**len(missing_mask)*(1/(3000*B**2*k**2))*I1_1
            I2_i = (1/(B*k*3000))*calculate_I2_i(X,data[i,missing_mask],h,len(missing_mask))
            I1.append(I1_i)
            I2.append(I2_i)
    else:
        st = time.time()
        distance_matrix = calculate_distances(data, complete_mask)
        d_time = time.time()-st
        for i in range(n):
            start_time = time.time()
            X,b_time = k_nearest_neighbors(i, B, distance_matrix, s_ratio, k , data ,missing_mask)
            
            B_time.append(b_time)
            
            I1_1 = calculate_I1_i(X,h)
            I1_i = (1/(2*h*np.pi**0.5))**len(missing_mask)*(1/(3000*B**2*k**2))*I1_1
            I2_i = (1/(B*k*3000))*calculate_I2_i(X,data[i,missing_mask],h,len(missing_mask))

            I1.append(I1_i)
            I2.append(I2_i)
    print('I1',sum(I1),'I2',sum(I2))
    return sum(I1)-2*sum(I2),B_time,d_time,time.time()-st_1

In [89]:
#generate data

def normalization(data):
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)
    ranges = max_vals - min_vals
    normalized_data = (data - min_vals) / ranges
    return normalized_data, min_vals, ranges
def generate_normalized_data(num_samples,vector):
    u = np.random.uniform(0,1,(num_samples,vector))+np.random.normal(0, 0.05,(num_samples,vector))
    x = 4*np.pi*u
    y = np.sum(np.sin(x),axis=1).reshape(-1,1)+np.random.normal(0,0.5,(num_samples,1))
    data = np.concatenate((x,y),axis=1)
    normalized_data,_,_ = normalization(data) 
    return normalized_data

In [90]:
#sample size (1000,3),B=15,s=0.7,k=50
data = generate_normalized_data(1000,2)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 15,50,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 0.39511689110507847 I2 0.44406495842066945
the time for finding B*K the neighbours 12.15130090713501
the time for calculating the distance 0.016994237899780273
the time for computing score 39.713834285736084
finding neighbours/all 0.3059714864021418
computing distance/all 0.00042791732919840605


In [86]:
#sample size (3000,3),B=15,s=0.7,k=10
data = generate_normalized_data(3000,2)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 15,10,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 1.2395887974756143 I2 1.1079681090519935
the time for finding B*K the neighbours 17.37077784538269
the time for calculating the distance 0.1239631175994873
the time for computing score 20.175918579101562
finding neighbours/all 0.8609658974028341
computing distance/all 0.006144112701162943


In [62]:
#sample size (3000,3),B=15,s=0.7,k=60
data = generate_normalized_data(3000,2)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 15, 60,0.1, np.array([0]),np.array([1,2]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 1.2195266402716165 I2 1.1049552781934853
the time for finding B*K the neighbours 107.02688431739807
the time for calculating the distance 0.14595556259155273
the time for computing score 224.65017366409302
finding neighbours/all 0.476415764883536
computing distance/all 0.0006497015346615846


In [63]:
#sample size (3000,7),B=15,s=0.7,k=60，缺失1维
data = generate_normalized_data(3000,6)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 15, 60,0.1, np.array([0]),np.array([1,2,3,4,5,6]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 1.1397598439113528 I2 1.106654167683386
the time for finding B*K the neighbours 105.58758282661438
the time for calculating the distance 0.253920316696167
the time for computing score 221.8547604084015
finding neighbours/all 0.4759311120132983
computing distance/all 0.0011445340015636249


In [64]:
#sample size (3000,7),B=15,s=0.7,k=60，缺失3维
data = generate_normalized_data(3000,6)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 15, 60,0.1, np.array([0,1,2]),np.array([3,4,5,6]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 2.1053218667792333 I2 1.307065542538779
the time for finding B*K the neighbours 107.28389358520508
the time for calculating the distance 0.21493244171142578
the time for computing score 227.80931544303894
finding neighbours/all 0.47093725459190083
computing distance/all 0.0009434752099291002


In [65]:
#sample size (3000,10),B=15,s=0.7,k=60，缺失3维
data = generate_normalized_data(3000,9)
score,B_time,s_time,all_time = calculate_score_restricted(data,0.7, 10, 30,0.1, np.array([0,1,2]),np.array([3,4,5,6,7,8,9]))
print('the time for finding B*K the neighbours',sum(B_time))
print('the time for calculating the distance',s_time)
print('the time for computing score',all_time)
print('finding neighbours/all',sum(B_time)/all_time)
print('computing distance/all',s_time/all_time)

I1 2.5557729965707234 I2 1.3303886811740806
the time for finding B*K the neighbours 54.217732429504395
the time for calculating the distance 0.23792362213134766
the time for computing score 65.49504947662354
finding neighbours/all 0.8278142067646771
computing distance/all 0.003632696272964375
